### 1. 문제정의/ 연구배경 : 마케팅
 - 펀드를 보유하지 않은 사람들에게 펀드를 판매하고싶다.
 - 현재 펀드 가입 여부 확인한다.
 - 펀드를 판매할 대상의 특성을 기반으로 펀드가입여부를 분류예측을 한다.

### 2. 데이터 수집

- MDIS (마이크로데이터 통합서비스, (재)한국통계진흥원) : MicroData Integreated Service
- https://mdis.mods.go.kr/index.do (회원가입 필요)
- https://mdis.mods.go.kr/mypage/extract/viewMyExtractDetailNew.do 
- MDIS 가계금융복지조사(2017년 이후) 가구 마스터(제공)

 - 데이터 Set 설명 참조
 - KDT/AI Precampus/20회차 파일설계서(공공용)_가계금융복지조사(2017년 이후)_가구마스터(제공)_2024(코드집포함).xlsx

In [2]:
import pandas as pd

df = pd.read_csv("2024_가구마스터.csv", encoding="cp949")
print(df.shape)
df.head(10)

(18314, 162)


,조사연도,MD제공용_가구고유번호,가중값,수도권여부,가구주_성별코드,가구원수,노인가구여부,조손가구여부,한부모가구여부,다문화가구여부(2012년~),...,원리금상환_가계지출감소항목코드,부채상환가능시기코드,가구주_은퇴여부,가구주_미은퇴_은퇴연령,가구주_미은퇴_최소생활비,가구주_미은퇴_적정생활비,가구주_미은퇴_노후준비상황코드,가구주_은퇴_은퇴연령,가구주_은퇴_적정생활비충당여부,가구주_은퇴_생활비재원마련코드(2014년~)
0,2024,10000162,967.170171,G1,2,1,G1,G2,G2,G2,...,NaN,NaN,2,NaN,0,0,NaN,76.0,4.0,4.0
1,2024,10000182,1077.062791,G1,1,2,G2,G2,G2,G2,...,NaN,NaN,1,70.0,200,250,4.0,NaN,NaN,NaN
2,2024,10000192,5002.471070,G1,2,1,G2,G2,G2,G2,...,NaN,NaN,1,60.0,200,300,5.0,NaN,NaN,NaN
3,2024,10000212,707.312221,G2,2,1,G1,G2,G2,G2,...,NaN,NaN,1,75.0,150,200,3.0,NaN,NaN,NaN
4,2024,10000222,1657.685816,G2,1,1,G2,G2,G2,G2,...,NaN,1.0,1,70.0,150,200,3.0,NaN,NaN,NaN
5,2024,10000262,891.869146,G2,2,2,G1,G2,G2,G2,...,NaN,NaN,2,NaN,0,0,NaN,69.0,3.0,2.0
6,2024,10000272,1860.372597,G2,1,4,G2,G2,G2,G2,...,NaN,NaN,2,NaN,0,0,NaN,60.0,3.0,2.0
7,2024,10000282,2664.164302,G2,1,4,G2,G2,G2,G2,...,4.0,3.0,1,70.0,200,200,4.0,NaN,NaN,NaN
8,2024,10000312,2801.485805,G1,1,5,G2,G2,G2,G2,...,NaN,NaN,1,65.0,100,200,2.0,NaN,NaN,NaN
9,2024,10000332,6435.823612,G1,2,2,G2,G2,G2,G2,...,1.0,1.0,1,65.0,300,500,3.0,NaN,NaN,NaN


### 3. 타겟 변수 설정

In [3]:
target_col = "자산_금융자산_저축_적립예치식저축_주식채권펀드금액"

df[target_col] = pd.to_numeric(df[target_col])
df["fund_holder"] = (df[target_col] > 0).astype(int)

df[["fund_holder", target_col]].head()


,fund_holder,자산_금융자산_저축_적립예치식저축_주식채권펀드금액
0,0,0
1,0,0
2,0,0
3,0,0
4,1,11100


In [4]:
df["fund_holder"].value_counts(normalize=True)


fund_holder
0    0.735284
1    0.264716
Name: proportion, dtype: float64

In [4]:
# 펀드 가입 비율 : 전체 고객의 26.47%

### 4. 사용할 변수 설정

 - 실습 편의상 변수 임의 선정

In [5]:
feature_cols = [
    "수도권여부",
    "소득5분위코드(보완)",
    "가구원수",
    "가구주_은퇴여부",
    "가구주_미은퇴_은퇴연령",
    "가구주_미은퇴_최소생활비",
    "가구주_미은퇴_적정생활비",
    "가구주_미은퇴_노후준비상황코드",
    "원리금연체여부",
    "30일이상원리금연체여부",
    "원리금상환_생계부담정도코드",
    "부채상환가능시기코드"

]

# 임시 dataframe
X = df[feature_cols].copy()

X.head()


,수도권여부,소득5분위코드(보완),가구원수,가구주_은퇴여부,가구주_미은퇴_은퇴연령,가구주_미은퇴_최소생활비,가구주_미은퇴_적정생활비,가구주_미은퇴_노후준비상황코드,원리금연체여부,30일이상원리금연체여부,원리금상환_생계부담정도코드,부채상환가능시기코드
0,G1,Q1,1,2,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,G1,Q2,2,1,70.0,200,250,4.0,NaN,NaN,NaN,NaN
2,G1,Q2,1,1,60.0,200,300,5.0,NaN,NaN,NaN,NaN
3,G2,Q1,1,1,75.0,150,200,3.0,NaN,NaN,NaN,NaN
4,G2,Q2,1,1,70.0,150,200,3.0,2.0,NaN,1.0,1.0


### 5. 데이터 전처리

In [6]:
# 수도권 여부 
df['수도권여부'].unique()

array(['G1', 'G2'], dtype=object)

In [7]:
# 소득5분위코드(보완) 
df['소득5분위코드(보완)'].unique()

array(['Q1', 'Q2', 'Q5', 'Q3', 'Q4'], dtype=object)

In [8]:
# 이진 변수

print(df['가구주_은퇴여부'].unique())
print(df['원리금연체여부'].unique())
print(df['30일이상원리금연체여부'].unique())

[2 1]
[nan  2.  1.]
[nan  2.  1.]


### 참고.

전처리 중 해당사항이 없는 대상자는 0으로 처리하는 전처리가 있습니다.

이러한 분포를 zero inflated distribution이라 하며,

해당 전처리는 프로젝트에서 자세하게 다룰 예정입니다.

In [9]:
# 수도권 여부 이진 인코딩
# 1 : 수도권
# 2 : 비수도권
df["수도권여부"] = (df["수도권여부"] == "G1").astype(int)

# 소득5분위코드(보완) 숫자 추출
# 1 : 1분위 (최저)
# 2 : 2분위
# 3 : 3분위
# 4 : 4분위
# 5 : 5분위 (최고)
income_map = {"Q1":1,"Q2":2,"Q3":3,"Q4":4,"Q5":5}
df["소득5분위코드(보완)"] = df["소득5분위코드(보완)"].map(income_map)

# 가구주_은퇴여부 : 기존 1,2 로 되어있던 범주형 데이터를 1,0으로 변환
# 데이터를 1,0 으로 일원화 (추후 동일작업 지속)
df["가구주_은퇴여부"] = df["가구주_은퇴여부"].replace({1:1, 2:0}).fillna(0)

#가구주_미은퇴_은퇴연령
# 은퇴 예상연령이므로 은퇴자/ 비대상은 결측치로 처리되었다. 따라서 zerofill 처리한다.
df["가구주_미은퇴_은퇴연령"] = df["가구주_미은퇴_은퇴연령"].fillna(0)

# 가구주_미은퇴_최소생활비
# 가구주_미은퇴_적정생활비
# 비해당은 결측이므로 0처리한다.
life_cols = [
    "가구주_미은퇴_최소생활비",
    "가구주_미은퇴_적정생활비"
]
df[life_cols] = df[life_cols].fillna(0)

# 가구주_미은퇴_노후준비상황코드
# 1 : 전혀 준비 안 됨
# 2 : 부족
# 3 : 보통
# 4 : 잘 준비
# 5 : 매우 잘 준비
df["가구주_미은퇴_노후준비상황코드"] = df["가구주_미은퇴_노후준비상황코드"].fillna(0)

# 원리금연체여부 : 기존 1,2를 1,0으로 변환
# 1 : 연체 경험 있음
# 2 : 없음
df["원리금연체여부"] = df["원리금연체여부"].replace({1:1, 2:0}).fillna(0)

# 30일이상원리금연체여부 : 기존 1,2를 1,0으로 변환
# 1 : 있음
# 2 : 없음
df["30일이상원리금연체여부"] = df["30일이상원리금연체여부"].replace({1:1, 2:0}).fillna(0)

# 원리금상환_생계부담정도코드
# 1 : 전혀 부담 안 됨
# 2 : 별로 부담 안 됨
# 3 : 보통
# 4 : 부담
# 5 : 매우 부담
df["원리금상환_생계부담정도코드"] = df["원리금상환_생계부담정도코드"].fillna(0)

# 부채상환가능시기코드
# 1 : 1년 이내
# 2 : 1~3년
# 3 : 3~5년
# 4 : 5년 이상
# 5 : 상환 불가능
df["부채상환가능시기코드"] = df["부채상환가능시기코드"].fillna(0)



### 6. 데이터 사용을 위한 데이터 타입 변환 : NN에 사용할 feature만 선택

In [9]:
# 데이터 사용을 위한 데이터 타입 변환

# NN에 사용할 feature만 선택
X = df[feature_cols].copy()

# 문자열/코드/혼합 타입 → 숫자형으로 강제 변환
X = X.apply(pd.to_numeric, errors="coerce")

# 결측치 처리 (이번 실습 기준)
X = X.fillna(0)

# dtype 확인 (전부 int 또는 float이어야 함)
X.dtypes


수도권여부               float64
소득5분위코드(보완)         float64
가구원수                  int64
가구주_은퇴여부              int64
가구주_미은퇴_은퇴연령        float64
가구주_미은퇴_최소생활비         int64
가구주_미은퇴_적정생활비         int64
가구주_미은퇴_노후준비상황코드    float64
원리금연체여부             float64
30일이상원리금연체여부        float64
원리금상환_생계부담정도코드      float64
부채상환가능시기코드          float64
dtype: object

In [10]:
# y 데이터 선택
y = df["fund_holder"].copy()
y

0        0
1        0
2        0
3        0
4        1
        ..
18309    0
18310    0
18311    0
18312    1
18313    0
Name: fund_holder, Length: 18314, dtype: int64

### 7. 데이터 분할 & 스케일링 

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train.values)   # numpy
X_test_s  = scaler.transform(X_test.values)        # numpy


### 8. NN 텐서 구조 설정
- NumPy / pandas 데이터를 PyTorch Tensor로 변환하고, 마지막 줄에서 학습 데이터의 형태(shape)를 확인하는 역할

In [13]:
# 입력 데이터 Tensor 변환 : PyTorch 모델이 사용 가능한 32비트 실수 Tensor로 변환→ 신경망 학습 시 가장 일반적으로 사용됨
X_train_t = torch.tensor(X_train_s, dtype=torch.float32)
X_test_t  = torch.tensor(X_test_s, dtype=torch.float32)

# 타깃(label) 데이터 Tensor 변환
# y_train.values : # y_train이 pandas Series일 경우,  # .values로 NumPy 배열 추출
# .reshape(-1, 1) : # 1차원 배열 (N,) → 2차원 (N, 1) 로 변환 ,  PyTorch의 회귀 모델 출력 형식에 맞추기 위함

y_train_t = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)
y_test_t  = torch.tensor(y_test.values.reshape(-1, 1), dtype=torch.float32)

X_train_t.shape, y_train_t.shape  # shape 확인


(torch.Size([14651, 12]), torch.Size([14651, 1]))

- 14651 : 학습 샘플 수,  12: 입력 특성(feature) 수, 1 : 예측할 값의 차원 (회귀)

### (복습 : 텐서 자동미분)

In [14]:
# Tensor 생성 + 자동미분 설정
# 값: x = 3.0 requires_grad=True :→ 이 Tensor에 대한 미분값(gradient)을 추적하겠다는 의미, → 역전파(backpropagation) 대상이 됨
# 기본값은 False : → 미분이 필요 없는 값은 계산 그래프에 포함되지 않음

x = torch.tensor(3.0, requires_grad=True)  # 자동미분 대상인지 여부

y = x**2 + 2*x  # 연산 수행 (
y.backward()
x.grad



tensor(8.)

- y를 x로 미분시 2x+2=3 ⇒2(3)+2=8

### (복습: PyTorch로 가장 기본적인 선형회귀(linear regression)를 학습)

In [15]:
import torch.nn as nn
import torch.optim as optim

x = torch.randn(100, 1)  # 데이터 생성
y = 3*x + 1

model = nn.Linear(1, 1) # 모델 정의 , 입력 1차원 → 출력 1차원, 내부 수식:𝑦^=𝑤𝑥+𝑏
criterion = nn.MSELoss()  # 손실 함수 & 옵티마이저
optimizer = optim.SGD(model.parameters(), lr=0.1)  # 옵티마이저: SGD (확률적 경사하강법)

# 학습 루프
for epoch in range(100):
    y_hat = model(x)  # 예측
    loss = criterion(y_hat, y)  # 손실 계산

    loss.backward()  # 역전파
    optimizer.step() # 파라미터 업데이트
    optimizer.zero_grad()  # gradient 초기화

# loss.item() :최종 평균 제곱 오차 --> 0에 가까울수록 학습이 잘 됨
# model.weight.item() : 학습된 기울기 w
# model.bias.item() : 학습된 절편 b

loss.item(), model.weight.item(), model.bias.item()


(2.8567371992277435e-13, 2.999999523162842, 1.000000238418579)

- (결과 해석)  y = 3*x + 1
- Loss ≈ 0 : 실제 직선을 거의 완벽히 학습
- weight ≈ 3 : 실제 기울기와 거의 동일
- bias ≈ 1 : 실제 절편과 거의 동일

### 9. NN 구조 생성

- 이진 분류(Binary Classification) 를 위한 MLP 모델 정의 + 클래스 불균형을 고려한 손실 함수 + Adam 옵티마이저 설정

In [16]:
# 모델 정의 (nn.Sequential)

model = nn.Sequential(
    nn.Linear(X_train_t.shape[1], 32),  # (1) 첫 번째 Linear
    nn.ReLU(),  # ReLU 활성화 함수
    nn.Linear(32, 1)  # 출력값: logit, 아직 sigmoid를 통과하지 않은 값
)

# 초기 가중치 설정
# 클래스 불균형을 고려한 손실 함수
pos_weight = torch.tensor([(len(y_train_t) - y_train_t.sum()) / y_train_t.sum()])  # y_train_t.sum() → 양성(1) 샘플 수
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # z = logit, σ(z) = sigmoid , w = pos_weight
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # 옵티마이저 (Adam), 학습률lr = 0.001


### 10. 모델 학습 시작

In [17]:
for epoch in range(200):
    model.train()

    logits = model(X_train_t)
    loss = criterion(logits, y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
# 학습 중 일정 주기마다(epoch 5번마다) 손실값(loss)을 출력하기 위한 디버깅/모니터링용 코드
    if (epoch + 1) % 5 == 0:  # epoch + 1 현재 학습이 몇 번째 epoch인지 출력
        print(epoch + 1, "loss:", loss.detach().item())  
# loss는 계산 그래프에 연결된 Tensor 그대로 출력해도 되지만, detach()는:계산 그래프에서 분리 gradient 추적 중단


5 loss: 1.0388785600662231
10 loss: 1.0223342180252075
15 loss: 1.0081273317337036
20 loss: 0.9961443543434143
25 loss: 0.9859639406204224
30 loss: 0.9772458076477051
35 loss: 0.9698388576507568
40 loss: 0.9634994268417358
45 loss: 0.9579651951789856
50 loss: 0.9530622363090515
55 loss: 0.9486385583877563
60 loss: 0.9445776343345642
65 loss: 0.9407840967178345
70 loss: 0.9371873736381531
75 loss: 0.9337414503097534
80 loss: 0.9304261803627014
85 loss: 0.9272423982620239
90 loss: 0.9241821765899658
95 loss: 0.921260416507721
100 loss: 0.9184830188751221
105 loss: 0.915870726108551
110 loss: 0.9135037064552307
115 loss: 0.9113175868988037
120 loss: 0.9093070030212402
125 loss: 0.9074552059173584
130 loss: 0.9057531952857971
135 loss: 0.9041970372200012
140 loss: 0.9027084708213806
145 loss: 0.9013116359710693
150 loss: 0.9000165462493896
155 loss: 0.898810863494873
160 loss: 0.8978166580200195
165 loss: 0.8969585299491882
170 loss: 0.8961686491966248
175 loss: 0.8954498767852783
180 loss

### 11. 모델 평가지표 확인
- 전체 흐름 요약
- model.eval()
  ↓
- no_grad()
  ↓
- logits = model(X_test)
-  ↓
- sigmoid → 확률
  ↓
- threshold → 예측 클래스

In [18]:
# 평가 모드로 전환
model.eval()
with torch.no_grad():  # Gradient 계산 비활성화 : 메모리 절약 + 추론 속도 향상
    test_logits = model(X_test_t)  # 모델 출력 (logits)
    prob = torch.sigmoid(test_logits)  # Sigmoid로 확률 변환
    pred = (prob >= 0.5).float()  # 이진 분류 결정 (Thresholding)


In [19]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test_t.numpy(), pred.numpy())
cm

print(classification_report(y_test_t.numpy(), pred.numpy(), digits=3))


              precision    recall  f1-score   support

         0.0      0.842     0.670     0.746      2693
         1.0      0.415     0.652     0.507       970

    accuracy                          0.665      3663
   macro avg      0.629     0.661     0.627      3663
weighted avg      0.729     0.665     0.683      3663



### 결과해석
- 펀드 마케팅 문제

- Accuracy는 중요한 지표가 아니다. 
- 마케팅 기준 recall 지표 성능을 확인하여 펀드 보유(가입)하는 고객의 특성을 확인하고 추후 잠재 고객을 발굴해 본다.